In [0]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove*.zip
# !ls
# !pwd

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [28]:
from collections import Counter
import os, re, pickle, random
from os.path import join as joinpath
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from nltk.stem.porter import PorterStemmer
tqdm.pandas()

seed = 0
random.seed(seed)
np.random.seed(seed)

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.utils import class_weight

import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils

from keras.layers import *
from keras.layers import TimeDistributed, Lambda
from keras.layers import Convolution1D, GlobalMaxPooling1D
from keras import callbacks  
from keras import backend as K
from keras.layers.advanced_activations import PReLU
from tensorflow.keras.models import load_model

from keras.preprocessing import sequence, text

from tensorflow.python.client import device_lib ; device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9326305557071037180, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 11015379599647306701
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13055032968422257549
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11330053735
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2670293756077328070
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [4]:
try:
  print(os.getcwd())
  os.chdir('drive/My Drive/Colab Notebooks/crowdflower-search-relevance')
  print('Changed directory')
  print(os.getcwd())
except:
  print('Cannot change directory')
  print(os.getcwd())

/content
Changed directory
/content/drive/My Drive/Colab Notebooks/crowdflower-search-relevance


In [5]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_train.head()

,id,query,product_title,product_description,median_relevance,relevance_variance
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,NaN,4,0.471
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471


In [6]:
y = df_train.median_relevance.values
Counter(y)

Counter({1: 774, 2: 1476, 3: 1737, 4: 6171})

In [7]:
encoders = {}
encoders['y'] = OneHotEncoder()
y = encoders['y'].fit_transform(df_train['median_relevance'].values.reshape(-1,1))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [8]:
df_train.fillna('', inplace=True)
df_test.fillna('', inplace=True)
df_train.head()

,id,query,product_title,product_description,median_relevance,relevance_variance
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,,4,0.471
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471


In [0]:
def remove_numbers(val):
    m = re.findall(r'([A-Za-z]\w+)', val, flags=re.I)
    return ' '.join(m)

stemmer = PorterStemmer()

def text_cleaner(x, append):
  x = BeautifulSoup(x).get_text(" ")
  x = re.sub("[^a-zA-Z0-9]", " ", x)
  x = ' '.join([stemmer.stem(w) for w in x.split()])
  if append:
    return ' '.join([append+w for w in x.split()])
  else:
    return x

In [10]:
for col in ['query', 'product_title', 'product_description']:
  if col == 'query':
    append = None
  elif col == 'product_title':
    append = None
  elif col == 'product_description':
    append = None
  df_train[col] = df_train[col].progress_apply(lambda x: text_cleaner(x, append))
  df_test[col] = df_test[col].progress_apply(lambda x: text_cleaner(x, append))

df_train['product_title'] = df_train[['product_title', 'product_description']].apply(lambda x: ' '.join(x), axis=1)
df_test['product_title'] = df_test[['product_title', 'product_description']].apply(lambda x: ' '.join(x), axis=1)
df_train.head()

 28%|██▊       | 2795/10158 [00:04<00:11, 646.74it/s]/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://i104.photobucket.com/albums/m175/champions_on_display/wincraft2013/januaryb/65497012.jpg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
 35%|███▍      | 3544/10158 [00:05<00:10, 607.58it/s]/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://i104.photobucket.com/albums/m175/champions_on_display/wincraft2013/januaryb/65516012.jpg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
 98%|█████████▊| 10003/10158 [00:14<00:00, 747.40it/s]/usr/local/lib

,id,query,product_title,product_description,median_relevance,relevance_variance
0,1,bridal shower decor,accent pillow with heart design red black red ...,red satin accent pillow embroid with a heart i...,1,0.000
1,2,led christma light,set of 10 batteri oper multi led train christm...,set of 10 batteri oper train christma light it...,4,0.000
2,4,projector,viewson pro8200 dlp multimedia projector,,4,0.471
3,5,wine rack,concept housewar WR 44526 solid wood ceil wall...,like a silent and sturdi tree the southern ent...,4,0.000
4,7,light bulb,wintergreen light christma led light bulb pack...,wtgr1011 featur nickel base 60 000 averag hour...,2,0.471


In [11]:
tk = text.Tokenizer(num_words=840000000000)

max_len_q = 5
max_len_p = 32
tk.fit_on_texts(list(df_train['query'].values.astype(str)) + list(df_train['product_title'].values.astype(str))) 

train_x1 = tk.texts_to_sequences(df_train['query'].values.astype(str))
train_x1 = sequence.pad_sequences(train_x1, maxlen=max_len_q)

train_x2 = tk.texts_to_sequences(df_train['product_title'].values.astype(str))
train_x2 = sequence.pad_sequences(train_x2, maxlen=max_len_p)

test_x1 = tk.texts_to_sequences(df_test['query'].values.astype(str))
test_x1 = sequence.pad_sequences(test_x1, maxlen=max_len_q)

test_x2 = tk.texts_to_sequences(df_test['product_title'].values.astype(str))
test_x2 = sequence.pad_sequences(test_x2, maxlen=max_len_p)

train_x1.shape

(10158, 5)

In [0]:
with open('data/tokenizer.pkl', 'wb') as f:
    pickle.dump(tk, f)

In [14]:
train_x1[0]

array([   0,    0, 1376,  722,  127], dtype=int32)

In [15]:
train_x2[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        451,  102,    5,  431,   22,  184,   36,  184, 1595,  451,  102,
       1380,    5,    3,  431,    8,   36, 2214,   88,   27,   88],
      dtype=int32)

In [16]:
word_index = tk.word_index

# ytrain_enc = np_utils.to_categorical(y)

emb_size = 50 #100
# file_path = 'data/glove.840B/glove.840B.%sd.txt'%emb_size
file_path = 'data/glove.6B/glove.6B.%sd.txt'%emb_size

def load_embed(file):
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    
    if file.split('.')[-1] == 'vec':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in tqdm(open(file, encoding='utf-8')) if len(o)>100)
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in tqdm(open(file, encoding='latin')))
        
    return embeddings_index

embeddings_index = load_embed(file_path)

400000it [00:07, 56202.48it/s]


In [17]:
print('Found %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((len(word_index) + 1, emb_size))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 22034/22034 [00:00<00:00, 516956.20it/s]

Found 400000 word vectors.


In [18]:
max_features = 200000
filter_length = 3
nb_filter = 64
pool_length = 4

model = Sequential()
print('Build model...')


Build model...


In [19]:
class ElmoEmbeddingLayer(Layer):
    def __init__(self, trainable=False, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)
    
ElmoEmbeddingLayer(trainable=False)

In [20]:
model1 = Sequential()
# model1.add(Input(shape=(1,), dtype="string"))
# model1.add(ElmoEmbeddingLayer(trainable=False)(input_text))
model1.add(Embedding(len(word_index) + 1,
                     emb_size,
                     weights=[embedding_matrix],
                     input_length=max_len_q,
                     trainable=False))

model1.add(TimeDistributed(Dense(emb_size, activation='relu')))
model1.add(Lambda(lambda x: K.sum(x, axis=1), output_shape=(emb_size,)))

model2 = Sequential()
# model2.add(ElmoEmbeddingLayer(trainable=False)())
model2.add(Embedding(len(word_index) + 1,
                     emb_size,
                     weights=[embedding_matrix],
                     input_length=max_len_p,
                     trainable=False))

model2.add(TimeDistributed(Dense(emb_size, activation='relu')))
model2.add(Lambda(lambda x: K.sum(x, axis=1), output_shape=(emb_size,)))

model3 = Sequential()
# model3.add(ElmoEmbeddingLayer(trainable=False)())
model3.add(Embedding(len(word_index) + 1,
                     emb_size,
                     weights=[embedding_matrix],
                     input_length=max_len_q,
                     trainable=False))
model3.add(Conv1D(filters=nb_filter,
                         kernel_size=filter_length,
                         padding='valid',
                         activation='relu',
                         strides=1))
model3.add(Dropout(0.2))

model3.add(Conv1D(filters=nb_filter,
                         kernel_size=filter_length,
                         padding='valid',
                         activation='relu',
                         strides=1))

model3.add(GlobalMaxPooling1D())
model3.add(Dropout(0.2))

model3.add(Dense(emb_size))
model3.add(Dropout(0.2))
model3.add(BatchNormalization())

model4 = Sequential()
# model4.add(ElmoEmbeddingLayer(trainable=False)())
model4.add(Embedding(len(word_index) + 1,
                     emb_size,
                     weights=[embedding_matrix],
                     input_length=max_len_p,
                     trainable=False))
model4.add(Conv1D(filters=nb_filter,
                         kernel_size=filter_length,
                         padding='valid',
                         activation='relu',
                         strides=1))
model4.add(Dropout(0.2))

model4.add(Conv1D(filters=nb_filter,
                         kernel_size=filter_length,
                         padding='valid',
                         activation='relu',
                         strides=1))

model4.add(GlobalMaxPooling1D())
model4.add(Dropout(0.2))

model4.add(Dense(emb_size))
model4.add(Dropout(0.2))
model4.add(BatchNormalization())

blstm_merge_mode = 'ave' #'concat'

model5 = Sequential()
# model5.add(ElmoEmbeddingLayer(trainable=False)())
model5.add(Embedding(len(word_index) + 1, emb_size, input_length=max_len_q, dropout=0.2))
model5.add(Bidirectional(LSTM(emb_size, dropout_W=0.2, dropout_U=0.2), merge_mode=blstm_merge_mode))

model6 = Sequential()
# model6.add(ElmoEmbeddingLayer(trainable=False)())
model6.add(Embedding(len(word_index) + 1, emb_size, input_length=max_len_p, dropout=0.2))
model6.add(Bidirectional(LSTM(emb_size, dropout_W=0.2, dropout_U=0.2), merge_mode=blstm_merge_mode))










Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:82: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:83: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(50, dropout=0.2, recurrent_dropout=0.2)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(50, dropout=0.2, recurrent_dropout=0.2)`


In [21]:
submodel_inputs = [model.input for model in [model1, model2, model3, model4, model5, model6]]
[print(model.output) for model in [model1, model2, model3, model4, model5, model6]]
submodel_outputs = [model.output for model in [model1, model2, model3, model4, model5, model6]]

# https://stackoverflow.com/questions/45979848/merge-2-sequential-models-in-keras
# mergedout = Concatenate(axis=1)(submodel_outputs)
mergedout1 = Average()([model1.output, model3.output, model5.output])
mergedout2 = Average()([model2.output, model4.output, model6.output])
mergedout = Multiply()([mergedout1, mergedout2])

print(mergedout1.shape)
print(mergedout2.shape)
print(mergedout.shape)

mergedout = BatchNormalization()(mergedout)

mergedout = Dense(emb_size)(mergedout)
mergedout = PReLU()(mergedout)
mergedout = Dropout(0.2)(mergedout)
mergedout = BatchNormalization()(mergedout)
mergedout = Dense(y.shape[1], activation='softmax')(mergedout)

cb_save_model = callbacks.ModelCheckpoint(joinpath('models', 'glove-nn-model.h5'),
                                                  monitor='val_acc', save_best_only=True, verbose=2)
cb_early_stopping = callbacks.EarlyStopping(monitor='val_acc', min_delta=0.001, patience=5, verbose=0, mode='auto')

all_callbacks = [cb_save_model, cb_early_stopping]

model = Model(submodel_inputs, mergedout)
model.compile(loss='categorical_crossentropy', 
                 optimizer='adam',
                 metrics=['accuracy'])

Tensor("lambda_1/Sum:0", shape=(?, 50), dtype=float32)
Tensor("lambda_2/Sum:0", shape=(?, 50), dtype=float32)
Tensor("batch_normalization_1/cond/Merge:0", shape=(?, 50), dtype=float32)
Tensor("batch_normalization_2/cond/Merge:0", shape=(?, 50), dtype=float32)
Tensor("bidirectional_1/truediv:0", shape=(?, 50), dtype=float32)
Tensor("bidirectional_2/truediv:0", shape=(?, 50), dtype=float32)
(?, 50)
(?, 50)
(?, 50)




In [22]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          13022        2989        2673          19        7360       11399
Swap:             0           0           0


In [0]:
def get_class_weights(y, one_hot=False):
    """Returns a dict of class weights for label encoded as well as one-hot encoded y."""
    if one_hot:
        y = np.argmax(y, axis=1)
    classes = np.unique(y)
    class_weights = class_weight.compute_class_weight('balanced', classes, y)
    return {c:w for c,w in zip(classes, class_weights)}

# class_weights = get_class_weights(y)
# class_weights

In [24]:
model.fit(x=[train_x1, train_x2, train_x1, train_x2, train_x1, train_x2], y=y, batch_size=32, epochs=100, 
             verbose=1, validation_split=0.1, shuffle=True, 
            #  class_weight=class_weights, 
             callbacks=all_callbacks)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 9142 samples, validate on 1016 samples
Epoch 1/100
9142/9142 [==============================] - 81s 9ms/step - loss: 1.3537 - acc: 0.4754 - val_loss: 1.0249 - val_acc: 0.6220

Epoch 00001: val_acc improved from -inf to 0.62205, saving model to models/siamese-model.h5
Epoch 2/100
9142/9142 [==============================] - 73s 8ms/step - loss: 1.0933 - acc: 0.5878 - val_loss: 0.9531 - val_acc: 0.6476

Epoch 00002: val_acc improved from 0.62205 to 0.64764, saving model to models/siamese-model.h5
Epoch 3/100
9142/9142 [==============================] - 75s 8ms/step - loss: 1.0137 - acc: 0.6097 - val_loss: 0.9317 - val_acc: 0.6447

Epoch 00003: val_acc did not improve from 0.64764
Epoch 4/100
9142/9142 [==============================] - 74s 8ms/step - loss: 0.9848 - acc: 0.6177 - val_loss: 0.9189 - val_acc: 0.6476

Epoch 00004: val_acc did not improve from 0.64764
Epoch 5/100
9142/9142

In [29]:
print('Making test predictions')
model = load_model('models/siamese-model.h5')
t_labels = model.predict([test_x1, test_x2, test_x1, test_x2, test_x1, test_x2])
t_labels = [int(x) for x in encoders['y'].inverse_transform(t_labels)]
print('Done!')

df_test['prediction'] = t_labels

submission_file_path = 'data/submission.csv'
df_test[['id', 'prediction']].to_csv(submission_file_path, index=False)
print('Submission saved!')

Making test predictions
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Done!
Submission saved!


In [0]:
# tf.enable_eager_execution()

# a = tf.convert_to_tensor(
#     [[1,2], [3,4]],
#     dtype=None,
#     name=None,
#     preferred_dtype=None
# )

# b = tf.convert_to_tensor(
#     [[1,2], [3,4]],
#     dtype=None,
#     name=None,
#     preferred_dtype=None
# )

# Dot(axes=1)([a,b])
# Multiply()([a,b])